## エラーログ用

In [0]:
%run /Workspace/Repos/prd_im_dlh/silver/EX_error_log_export

## カレンダーマスタから取得

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from pyspark.sql.functions import to_date, lit, date_sub, add_months, trunc, last_day
import pytz

try:
    # Sparkセッションの作成
    spark = SparkSession.builder \
        .appName("select_carendar") \
        .getOrCreate()

    #現在日時を取得
    jst = pytz.timezone('Asia/Tokyo')
    date = datetime.now(jst)
    seven_days_ago = date - timedelta(days=7)
    formatted_date = seven_days_ago.strftime('%Y-%m-%d')

    #カレンダーマスタからデータ取得
    result_df = spark.sql(
                f"""
                select * from prd_im_dlh.silver.dim_calendar
                where base_date = '{date}'
                """
                )

    #カレンダーマスタ
    base_date=result_df.collect()[0]['base_date']
    # week = result_df.collect()[0]['weeks']
    week_start_date = result_df.collect()[0]['week_start_date']
    same_day_last_year = result_df.collect()[0]['same_day_last_year']
    week_year = result_df.collect()[0]['week_year']

    # 週数用
    result_df = spark.sql(
                f"""
                select * from prd_im_dlh.silver.dim_calendar
                where base_date = '{formatted_date}'
                """
                )
    week = result_df.collect()[0]['weeks']

    #前日
    yesterday = (datetime.now(jst) - timedelta(days=1)).date()
    #レポート集計期間
    report_from = week_start_date - timedelta(days=7)
    report_to = week_start_date - timedelta(days=1)

    #-1Wレポート集計期間
    report_from_1w = week_start_date - timedelta(days=14)
    report_to_1w = week_start_date - timedelta(days=8)

    #-2Wレポート集計期間
    report_from_2w = week_start_date - timedelta(days=21)
    report_to_2w = week_start_date - timedelta(days=15)

    #-3Wレポート集計期間
    report_from_3w = week_start_date - timedelta(days=28)
    report_to_3w = week_start_date - timedelta(days=22)

    #前年レポート集計期間
    #カレンダーマスタからデータ取得
    result_1y_df = spark.sql(
                f"""
                select * from prd_im_dlh.silver.dim_calendar
                where base_date = '{same_day_last_year}'
                """
                )
    week_start_date_1y = result_1y_df.collect()[0]['week_start_date']
    report_from_1y = week_start_date_1y - timedelta(days=7)
    report_to_1y = week_start_date_1y - timedelta(days=1)
    seven_days_ago = date_sub(to_date(lit(week_start_date)), 7)

    # 2. 基準日から当月の1日と末日を取得
    pre_month = add_months(seven_days_ago, 0)
    pre_month_start = trunc(pre_month, "MM")
    pre_month_end = last_day(pre_month)

    # 2. 基準日から当月の1日と末日を取得
    this_month = add_months(seven_days_ago, 1)
    this_month_start = trunc(this_month, "MM")
    this_month_end = last_day(this_month)

    # 3. 次月の1日と末日を取得
    next_month = add_months(seven_days_ago, 2)
    next_month_start = trunc(next_month, "MM")
    next_month_end = last_day(next_month)

    # 4. DataFrameで評価しPythonのdate型に変換
    date_df = spark.range(1).select(
        pre_month_start.alias("pre_start"),
        pre_month_end.alias("pre_end"),
        this_month_start.alias("this_start"),
        this_month_end.alias("this_end"),
        next_month_start.alias("next_start"),
        next_month_end.alias("next_end")
    )
    row = date_df.collect()[0]
    pre_first_day = date_df.collect()[0]["pre_start"]
    pre_last_day = date_df.collect()[0]["pre_end"]
    current_first_day = date_df.collect()[0]["this_start"]
    current_last_day = date_df.collect()[0]["this_end"]
    next_first_day = date_df.collect()[0]["next_start"]
    next_last_day = date_df.collect()[0]["next_end"]


    # 出力確認

    # 変数に格納
    print(f"年: {week_year}")
    print(f"週数: {week}")
    print(f"昨日: {yesterday}")
    print(f"当週開始日: {report_from}")
    print(f"当週終了日: {report_to}")
    print(f"前週開始日: {report_from_1w}")
    print(f"前週終了日: {report_to_1w}")
    print(f"-2W開始日: {report_from_2w}")
    print(f"-2W終了日: {report_to_2w}")
    print(f"-3W開始日: {report_from_3w}")
    print(f"-3W終了日: {report_to_3w}")
    print(f"前年開始日: {report_from_1y}")
    print(f"前年終了日: {report_to_1y}")
    print(f"前月開始日: {pre_first_day}")
    print(f"前月終了日: {pre_last_day}")
    print(f"当月開始日: {current_first_day}")
    print(f"当月終了日: {current_last_day}")
    print(f"次月開始日: {next_first_day}")
    print(f"次月終了日: {next_last_day}")

    print(':')
    print('base_date')
    print(base_date)
    print('week')
    print(week)
    print('week_start_date')
    print(week_start_date)
    print('same_day_last_year')
    print(same_day_last_year)
    print('week_year')
    print(week_year)
    print('yesterday')
    print(yesterday)
    print('report_from')
    print(report_from)
    print('report_to')
    print(report_to)
    print('report_from_1w')
    print(report_from_1w)
    print('report_to_1w')
    print(report_to_1w)
    print('report_from_2w')
    print(report_from_2w)
    print('report_to_2w')
    print(report_to_2w)
    print('report_from_3w')
    print(report_from_3w)
    print('report_to_3w')
    print(report_to_3w)
    print('report_from_1y')
    print(report_from_1y)
    print('report_to_1y')
    print(report_to_1y)
    print('pre_first_day')
    print(pre_first_day)
    print('pre_last_day')
    print(pre_last_day)
    print('current_first_day')
    print(current_first_day)
    print('current_last_day')
    print(current_last_day)
    print('next_first_day')
    print(next_first_day)
    print('next_last_day')
    print(next_last_day)

# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

## 販売形態　総件数算出

In [0]:
try:      
    #アウトレットの総件数算出
    outlet_df = spark.sql(
        f"""
            select
            count(distinct s.id) as outlet_all
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from}')
            and cast(st.sales_date as date) <= date('{report_to}')
            and s.shop_class_code = '07'
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    outlet_all = outlet_df.collect()[0]["outlet_all"]
    display(outlet_df)

    # 催事の総件数算出
    event_df = spark.sql(
        f"""
            select
            COALESCE(count(distinct s.id),0) as event_all
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from}')
            and cast(st.sales_date as date) <= date('{report_to}')
            and s.shop_class_code = '70'
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    results = event_df.collect()
    event_all = results[0]["event_all"] if results else 0
    display(event_df)

    # プロパーの総件数算出
    proper_df = spark.sql(
        f"""
            select
            count(distinct s.id) as proper_all
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from}')
            and cast(st.sales_date as date) <= date('{report_to}')
            and s.shop_class_code not in ('07', '70')
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    proper_all = proper_df.collect()[0]["proper_all"]
    display(proper_df)

    # 併売の総件数算出
    both_all = outlet_all + event_all + proper_all
    print(both_all)

# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

## 販売形態　総件数算出（去年）

In [0]:
try:    
    #アウトレットの総件数算出
    outlet_1y_df = spark.sql(
        f"""
            select
            count(distinct s.id) as outlet_all_last_year
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from_1y}')
            and cast(st.sales_date as date) <= date('{report_to_1y}')
            and s.shop_class_code = '07'
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    outlet_all_last_year = outlet_1y_df.collect()[0]["outlet_all_last_year"]
    display(outlet_1y_df)

    # 催事の総件数算出
    event_1y_df = spark.sql(
        f"""
            select
            COALESCE(count(distinct s.id),0) as event_all_last_year
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from_1y}')
            and cast(st.sales_date as date) <= date('{report_to_1y}')
            and s.shop_class_code = '70'
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    results = event_1y_df.collect()
    event_all_last_year = results[0]["event_all_last_year"] if results else 0
    display(event_1y_df)

    # プロパーの総件数算出
    proper_1y_df = spark.sql(
        f"""
            select
            count(distinct s.id) as proper_all_last_year
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= date('{report_from_1y}')
            and cast(st.sales_date as date) <= date('{report_to_1y}')
            and s.shop_class_code not in ('07', '70')
            group by
            case 
                when s.shop_class_code = '07' then 'アウトレット'
                when s.shop_class_code = '70' then '催事'
            else 'プロパー'
            end
        """
    )
    proper_all_last_year = proper_1y_df.collect()[0]["proper_all_last_year"]
    display(proper_1y_df)

    # 併売の総件数算出
    both_all_last_year = outlet_all_last_year + event_all_last_year + proper_all_last_year
    print(both_all_last_year)
    
# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

## 店舗の総件数算出

In [0]:
try:    
    # 店舗の総件数を取得
    shop_df = spark.sql(
        f"""
            select
            count(distinct id) as shop_all
            from
            prd_im_dlh.silver.fact_sales_transaction st
                join prd_im_dlh.silver.dim_shop s
                on s.id = st.shop_id
            where
            st.company_code in ('10', '11')
            and s.shop_class_code not in ('09', '10')
            and cast(st.sales_date as date) >= ('{report_from}')
            and cast(st.sales_date as date) <= date('{report_to}')
        """
    )
    shop_all = shop_df.collect()[0]["shop_all"]




    # 前年店舗の総件数を取得
    ly_shop_df = spark.sql(
        f"""
            select 
                count(distinct id) as shop_all_last_year
            from prd_im_dlh.silver.fact_sales_transaction st
            join prd_im_dlh.silver. dim_shop s on s.id = st.shop_id
            where st.company_code in ('10', '11')
                and s.shop_class_code not in ('09', '10')
                and cast(st.sales_date as date) >= date('{report_from_1y}')
                and cast(st.sales_date as date) <= date('{report_to_1y}')
        """
    )
    shop_all_last_year = ly_shop_df.collect()[0]["shop_all_last_year"]


    print(shop_all)
    print(shop_all_last_year)

# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

## 売上高の総合計算出

In [0]:
try:
    #売上高の総合計を取得
    amount_df = spark.sql(
                f"""
                    SELECT
                        sum(sd.amount) as total_amount
                        FROM prd_im_dlh.silver.fact_sales_detail sd
                        left join prd_im_dlh.silver.dim_product p
                        on sd.product_id = p.id
                        WHERE sd.company_code in ('10', '11')
                        AND sd.pos_category_code not in ('99', '98')
                        and cast(sd.sales_date as date) >= date('{report_from}')
                        and cast(sd.sales_date as date) <= date('{report_to}')
                        AND p.large_class_code in ('01','02','03','04')
                """
                )
    total_amount = amount_df.collect()[0]['total_amount']

    print(total_amount)
    
# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

## 既存データ削除

In [0]:
try:
    spark.sql(
        f"""
        delete from prd_im_dlh.gold.weekly_sales_ranking_report1_header_all_2
        where report_from = '{report_from}'
        and report_to = '{report_to}'
        """
    )
# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")

# SKUを型番ごとに圧縮

In [0]:
try:    
    header_df = spark.sql(f"""
    with top as (
            SELECT
                row_number() over(order by gd.amount desc) as rank,
                gd.country as country,
                gd.po_category as po_category,
                gd.sales_qty_3w as sales_qty_3w,
                gd.sales_qty_2w as sales_qty_2w,
                gd.sales_qty_1w as sales_qty_1w,
                gd.qty as qty,
                gd.amount as amount,
                gd.qty_last_year as qty_last_year,
                gd.shop_inventory_qty as shop_inventory_qty,
                gd.warehouse_inventory_qty as warehouse_inventory_qty,
                gd.total_inventory_qty as total_inventory_qty,
                gd.in_progress_stock as in_progress_stock,
                gd.pre_month_order as pre_month_order,
                gd.current_month_order as current_month_order,
                gd.next_month_order as next_month_order,
                sf_newcomer.newcomer_quantity as sf_newcomer_n_quantity,
                sf_newcomer.quantity as sf_newcomer_quantity,
                gd.woman_10s_cnt + gd.woman_20s_cnt + gd.woman_30s_cnt + gd.woman_40s_cnt + gd.woman_50s_cnt + gd.woman_60s_cnt as woman_cnt,
                coalesce(lens_cnt.nonprescription_color_cnt,0) + coalesce(lens_cnt.nonprescription_ph_cnt,0)+ coalesce(lens_cnt.nonprescription_uv_cnt,0)+ coalesce(lens_cnt.prescription_color_cnt,0)+ coalesce(lens_cnt.prescription_ph_cnt,0)+ coalesce(lens_cnt.prescription_uv_cnt,0)+ coalesce(sg_uv.quantity,0)as lens_uv_cnt,
                coalesce(lens_cnt.nonprescription_quantity,0)-coalesce(sg_p.quantity,0) as nonprescription_quantity
            FROM
            (
                SELECT
                    gd.country as country,
                    min(gd.po_category) as po_category,
                    min(gd.large_class) as large_class,
                    min(gd.discontinued) as discontinued,
                    min(gd.target) as target,
                    min(gd.budget_actual_category) as budget_actual_category,
                    gd.model_no as model_no,
                    min(gd.material) as material,
                    min(gd.lens_shape) as lens_shape,
                    min(gd.current_price) as current_price,
                    date_diff('{report_to}' , min(gd.release_date)) / 7.0 as seles_week,
                    min(gd.series_name) as series_name,
                    sum(gd.sales_qty_3w) as sales_qty_3w,
                    sum(gd.sales_qty_2w) as sales_qty_2w,
                    sum(gd.sales_qty_1w) as sales_qty_1w,
                    sum(gd.qty) as qty,
                    sum(gd.amount) as amount,
                    sum(gd.qty_last_year) as qty_last_year,
                    sum(gd.shop_inventory_qty) as shop_inventory_qty,
                    sum(gd.warehouse_inventory_qty) as warehouse_inventory_qty,
                    sum(gd.total_inventory_qty) as total_inventory_qty,
                    sum(gd.in_progress_stock) as in_progress_stock,
                    sum(gd.pre_month_order) as pre_month_order,
                    sum(gd.current_month_order) as current_month_order,
                    sum(gd.next_month_order) as next_month_order,
                    sum(gd.man_10s_cnt) as man_10s_cnt,
                    sum(gd.man_20s_cnt) as man_20s_cnt,
                    sum(gd.man_30s_cnt) as man_30s_cnt,
                    sum(gd.man_40s_cnt) as man_40s_cnt,
                    sum(gd.man_50s_cnt) as man_50s_cnt,
                    sum(gd.man_60s_cnt) as man_60s_cnt,
                    sum(gd.woman_10s_cnt) as woman_10s_cnt,
                    sum(gd.woman_20s_cnt) as woman_20s_cnt,
                    sum(gd.woman_30s_cnt) as woman_30s_cnt,
                    sum(gd.woman_40s_cnt) as woman_40s_cnt,
                    sum(gd.woman_50s_cnt) as woman_50s_cnt,
                    sum(gd.woman_60s_cnt) as woman_60s_cnt,
                    sum(gd.unknown_cnt) as unknown_cnt,
                    min(gd.frame_height) as frame_height,
                    min(gd.lens_width) as lens_width,
                    min(gd.finishing_height) as finishing_height,
                    min(gd.finishing_width) as finishing_width,
                    min(gd.release_date) as release_date,
                    sum(gd.order_qty) as order_qty,
                    sum(gd.bad_inventory_qty) as bad_inventory_qty
                from
                    prd_im_dlh.gold.weekly_sales_ranking_report2_detail_all gd
                WHERE
                    cast(gd.report_from as date) = date('{report_from}')
                    AND cast(gd.report_to as date)= date('{report_to}')
                GROUP BY
                    gd.model_no,
                    gd.country
            )gd
            LEFT JOIN
            (
                select
                    sl.frame_model_no as model_no,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' THEN sl.frame_quantity ELSE 0 END) AS prescription_quantity,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' THEN sl.frame_quantity ELSE 0 END) AS nonprescription_quantity,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' and p.budget_actual_name in ('0801_機能なし', null) THEN sl.frame_quantity ELSE 0 END) AS prescription_nonfunction_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' and p.budget_actual_name = '0802_BLC' THEN sl.frame_quantity ELSE 0 END) AS prescription_blc_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' and p.budget_actual_name = '0803_カラー' THEN sl.frame_quantity ELSE 0 END) AS prescription_color_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' and p.budget_actual_name = '0804_調光' THEN sl.frame_quantity ELSE 0 END) AS prescription_ph_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '01度あり' and p.budget_actual_name = '0805_UV' THEN sl.frame_quantity ELSE 0 END) AS prescription_uv_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' and p.budget_actual_name in ('0801_機能なし', null) THEN sl.frame_quantity ELSE 0 END) AS nonprescription_nonfunction_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' and p.budget_actual_name = '0802_BLC' THEN sl.frame_quantity ELSE 0 END) AS nonprescription_blc_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' and p.budget_actual_name = '0803_カラー' THEN sl.frame_quantity ELSE 0 END) AS nonprescription_color_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' and p.budget_actual_name = '0804_調光' THEN sl.frame_quantity ELSE 0 END) AS nonprescription_ph_cnt,
                    SUM(CASE WHEN sl.is_prescribed = '02度なし' and p.budget_actual_name = '0805_UV' THEN sl.frame_quantity ELSE 0 END) AS nonprescription_uv_cnt

                from
                    prd_im_dlh.silver.fact_pos_sf_line sl
                    left join prd_im_dlh.silver.dim_product p
                        on p.product_code = coalesce(sl.lens_product_code_01, sl.lens_product_code_02)
                        and p.company_code = '11'
                where
                    cast(sl.date as date) >= date('{report_from}')
                    and cast(sl.date as date) <= date('{report_to}')
                GROUP BY
                    sl.frame_model_no
            )lens_cnt
            ON gd.model_no = lens_cnt.model_no
            
            ------------★★★★★SF新規率のパーツ　quantity/ newcomer_quantity　 ★★★★★---------------
            LEFT JOIN
            (
                select
                sl.frame_model_no as model_no,
                sum(sl.frame_quantity) as quantity,
                sum(case when sl.repeat_name = '01新規' then sl.frame_quantity else 0 end) as newcomer_quantity
                    
                from 
                prd_im_dlh.silver.fact_pos_sf_line sl
                    left join prd_im_dlh.silver.dim_product p 
                    on p.product_code = COALESCE(sl.lens_product_code_01, sl.lens_product_code_02) 
                    and p.company_code = left(sl.shop_code, 2)
                where
                    cast(sl.date as date) >= date('{report_from}')
                    and cast(sl.date as date) <= date('{report_to}')
                    and sl.shop_category in ('実店舗', 'オンラインストア')
                    and sl.repeat_name <> '03不明'
                group by
                    sl.frame_model_no
            )sf_newcomer
            on gd.model_no = sf_newcomer.model_no
            ------------★★★★★ サングラス用 UV ★★★★★---------------
            LEFT JOIN
            (
                select 
                    sf.frame_model_no as model_no,
                    sum(sf.frame_quantity) as quantity
                from 
                    prd_im_dlh.silver.fact_pos_sf_line sf
                    left join prd_im_dlh.silver.dim_product p
                        on p.product_code = sf.frame_product_code
                        and p.company_code = '11'
                where p.large_class_code = '02'	--サングラス
                    and lens_quantity_01 = '0' 
                    and lens_quantity_02 = '0' --レンズ購入無し
                    and cast(sf.date as date) >= date('{report_from}')
                    and cast(sf.date as date) <= date('{report_to}')
                GROUP BY
                    sf.frame_model_no
            )sg_uv
            on gd.model_no = sg_uv.model_no
            ------------★★★★★ サングラス用 度あり ★★★★★---------------
            LEFT JOIN
            (
                select 
                    sf.frame_model_no as model_no,
                    sum(sf.frame_quantity) as quantity
                from 
                    prd_im_dlh.silver.fact_pos_sf_line sf
                    left join prd_im_dlh.silver.dim_product p
                        on p.product_code = sf.frame_product_code
                        and p.company_code = '11'
                where (
                        (p.large_class_code = '03' and p.budget_actual_code in ('302', '305')) 
                            and (lens_quantity_01 = 0 and lens_quantity_02 = 0 )
                        )  --近用パッケージでレンズ購入なしor度付きレンズ
                    and cast(sf.date as date) >= date('{report_from}')
                    and cast(sf.date as date) <= date('{report_to}')
                GROUP BY
                    sf.frame_model_no
            )sg_p
            on gd.model_no = sg_p.model_no
    )

    select
        '01' as sort_number,
        top_no,
        country as country,
        '{week}' as weeks,
        '{report_from}' as report_from,
        '{report_to}' as report_to,
        sales_qty_3w,
        sales_qty_2w,
        sales_qty_1w,
        qty as qty,
        (qty/sales_qty_1w)*100 as qty_wow,
        qty - sales_qty_1w as qty_wow_diff,
        CASE
            when po_category = 'Proper Only' then qty/'{proper_all}'
            when po_category = 'All Store' then qty/'{both_all}' 
            when po_category = 'Outlet Only' then qty/'{outlet_all}' 
            else  0
        end as per_shop_qty,
        amount as amount,
        amount /'{total_amount}'*100 as revenue_ratio,
        '-' as cumulative_ratio,
        qty_last_year,
        (qty/qty_last_year)*100 as gross_yoy,
        CASE
                when po_category = 'Proper Only' then qty_last_year/'{proper_all_last_year}'
                when po_category = 'All Store' then qty_last_year/'{both_all_last_year}' 
                when po_category = 'Outlet Only' then qty_last_year/'{outlet_all_last_year}' 
                else  0
            end as per_shop_qty_last_year,
        CASE
                when po_category = 'Proper Only' then (qty/'{proper_all}')/(qty_last_year/'{proper_all_last_year}')*100
                when po_category = 'All Store' then (qty/'{both_all}')/(qty_last_year/'{both_all_last_year}')*100 
                when po_category = 'Outlet Only' then (qty/'{outlet_all}')/(qty_last_year/'{outlet_all_last_year}')*100 
                else  0
            end as per_shop_qty_yoy,
        shop_inventory_qty as shop_inventory_qty,
        shop_inventory_qty/qty as shop_inventory_qty_week,
        warehouse_inventory_qty as warehouse_inventory_qty,
        warehouse_inventory_qty/qty as warehouse_inventory_qty_week,
        total_inventory_qty as total_inventory_qty,
        total_inventory_qty/qty as total_inventory_qty_week,
        in_progress_stock as in_progress_stock,
        in_progress_stock/qty as in_progress_week,
        pre_month_order as pre_month_order,
        current_month_order as current_month_order,
        next_month_order as next_month_order,
        newcomer_quantity/quantity*100 as sf_newcomer_ratio,
        customer_woman_cnt/qty*100 as customer_woman_ratio,
        uv_lens_cnt/qty*100 as uv_lens_ratio,
        nonprescription_quantity/qty*100 as nonprescription_ratio,
        -- 作成日時
        current_timestamp() AS created_datetime,
        -- 作成者
        NULL AS created_user,
        -- 更新日時
        current_timestamp() AS updated_datetime,
        -- 更新者
        NULL AS updated_user

        
    from
    (
        select
            
            'TOP10' as top_no,
            country as country,
            min(po_category) as po_category,
            sum(sales_qty_3w) as sales_qty_3w,
            sum(sales_qty_2w) as sales_qty_2w,
            sum(sales_qty_1w) as sales_qty_1w,
            sum(qty) as qty,
            sum(amount) as amount,
            sum(qty_last_year) as qty_last_year,
            sum(shop_inventory_qty) as shop_inventory_qty,
            sum(warehouse_inventory_qty) as warehouse_inventory_qty,
            sum(total_inventory_qty) as total_inventory_qty,
            sum(in_progress_stock) as in_progress_stock,
            sum(pre_month_order) as pre_month_order,
            sum(current_month_order) as current_month_order,
            sum(next_month_order) as next_month_order,
            sum(sf_newcomer_n_quantity) as newcomer_quantity,
            sum(sf_newcomer_quantity) as quantity,
            sum(woman_cnt) as customer_woman_cnt,
            sum(lens_uv_cnt) as uv_lens_cnt,
            sum(nonprescription_quantity) as nonprescription_quantity
        from
            top
        where
            rank <= 10
        group by
            'TOP10',
            country
    )



    union all
    select
        '02' as sort_number,
        top_no,
        country as country,
        '{week}' as weeks,
        '{report_from}' as report_from,
        '{report_to}' as report_to,
        sales_qty_3w,
        sales_qty_2w,
        sales_qty_1w,
        qty as qty,
        (qty/sales_qty_1w)*100 as qty_wow,
        qty - sales_qty_1w as qty_wow_diff,
        CASE
            when po_category = 'Proper Only' then qty/'{proper_all}'
            when po_category = 'All Store' then qty/'{both_all}' 
            when po_category = 'Outlet Only' then qty/'{outlet_all}' 
            else  0
        end as per_shop_qty,
        amount as amount,
        amount /'{total_amount}'*100 as revenue_ratio,
        '-' as cumulative_ratio,
        qty_last_year,
        (qty/qty_last_year)*100 as gross_yoy,
        CASE
                when po_category = 'Proper Only' then qty_last_year/'{proper_all_last_year}'
                when po_category = 'All Store' then qty_last_year/'{both_all_last_year}' 
                when po_category = 'Outlet Only' then qty_last_year/'{outlet_all_last_year}' 
                else  0
            end as per_shop_qty_last_year,
        CASE
                when po_category = 'Proper Only' then (qty/'{proper_all}')/(qty_last_year/'{proper_all_last_year}')*100
                when po_category = 'All Store' then (qty/'{both_all}')/(qty_last_year/'{both_all_last_year}')*100 
                when po_category = 'Outlet Only' then (qty/'{outlet_all}')/(qty_last_year/'{outlet_all_last_year}')*100 
                else  0
            end as per_shop_qty_yoy,
        shop_inventory_qty as shop_inventory_qty,
        shop_inventory_qty/qty as shop_inventory_qty_week,
        warehouse_inventory_qty as warehouse_inventory_qty,
        warehouse_inventory_qty/qty as warehouse_inventory_qty_week,
        total_inventory_qty as total_inventory_qty,
        total_inventory_qty/qty as total_inventory_qty_week,
        in_progress_stock as in_progress_stock,
        in_progress_stock/qty as in_progress_week,
        pre_month_order as pre_month_order,
        current_month_order as current_month_order,
        next_month_order as next_month_order,
        newcomer_quantity/quantity*100 as sf_newcomer_ratio,
        customer_woman_cnt/qty*100 as customer_woman_ratio,
        uv_lens_cnt/qty*100 as uv_lens_ratio,
        nonprescription_quantity/qty*100 as nonprescription_ratio,
        -- 作成日時
        current_timestamp() AS created_datetime,
        -- 作成者
        NULL AS created_user,
        -- 更新日時
        current_timestamp() AS updated_datetime,
        -- 更新者
        NULL AS updated_user

        
    from
    (
        select
            
            'TOP30' as top_no,
            country as country,
            min(po_category) as po_category,
            sum(sales_qty_3w) as sales_qty_3w,
            sum(sales_qty_2w) as sales_qty_2w,
            sum(sales_qty_1w) as sales_qty_1w,
            sum(qty) as qty,
            sum(amount) as amount,
            sum(qty_last_year) as qty_last_year,
            sum(shop_inventory_qty) as shop_inventory_qty,
            sum(warehouse_inventory_qty) as warehouse_inventory_qty,
            sum(total_inventory_qty) as total_inventory_qty,
            sum(in_progress_stock) as in_progress_stock,
            sum(pre_month_order) as pre_month_order,
            sum(current_month_order) as current_month_order,
            sum(next_month_order) as next_month_order,
            sum(sf_newcomer_n_quantity) as newcomer_quantity,
            sum(sf_newcomer_quantity) as quantity,
            sum(woman_cnt) as customer_woman_cnt,
            sum(lens_uv_cnt) as uv_lens_cnt,
            sum(nonprescription_quantity) as nonprescription_quantity
        from
            top
        where
            rank <= 30
        group by
            'TOP10',
            country
    )

    union all
    select
        '03' as sort_number,
        top_no,
        country as country,
        '{week}' as weeks,
        '{report_from}' as report_from,
        '{report_to}' as report_to,
        sales_qty_3w,
        sales_qty_2w,
        sales_qty_1w,
        qty as qty,
        (qty/sales_qty_1w)*100 as qty_wow,
        qty - sales_qty_1w as qty_wow_diff,
        CASE
            when po_category = 'Proper Only' then qty/'{proper_all}'
            when po_category = 'All Store' then qty/'{both_all}' 
            when po_category = 'Outlet Only' then qty/'{outlet_all}' 
            else  0
        end as per_shop_qty,
        amount as amount,
        amount /'{total_amount}'*100 as revenue_ratio,
        '-' as cumulative_ratio,
        qty_last_year,
        (qty/qty_last_year)*100 as gross_yoy,
        CASE
                when po_category = 'Proper Only' then qty_last_year/'{proper_all_last_year}'
                when po_category = 'All Store' then qty_last_year/'{both_all_last_year}' 
                when po_category = 'Outlet Only' then qty_last_year/'{outlet_all_last_year}' 
                else  0
            end as per_shop_qty_last_year,
        CASE
                when po_category = 'Proper Only' then (qty/'{proper_all}')/(qty_last_year/'{proper_all_last_year}')*100
                when po_category = 'All Store' then (qty/'{both_all}')/(qty_last_year/'{both_all_last_year}')*100 
                when po_category = 'Outlet Only' then (qty/'{outlet_all}')/(qty_last_year/'{outlet_all_last_year}')*100 
                else  0
            end as per_shop_qty_yoy,
        shop_inventory_qty as shop_inventory_qty,
        shop_inventory_qty/qty as shop_inventory_qty_week,
        warehouse_inventory_qty as warehouse_inventory_qty,
        warehouse_inventory_qty/qty as warehouse_inventory_qty_week,
        total_inventory_qty as total_inventory_qty,
        total_inventory_qty/qty as total_inventory_qty_week,
        in_progress_stock as in_progress_stock,
        in_progress_stock/qty as in_progress_week,
        pre_month_order as pre_month_order,
        current_month_order as current_month_order,
        next_month_order as next_month_order,
        newcomer_quantity/quantity*100 as sf_newcomer_ratio,
        customer_woman_cnt/qty*100 as customer_woman_ratio,
        uv_lens_cnt/qty*100 as uv_lens_ratio,
        nonprescription_quantity/qty*100 as nonprescription_ratio,
        -- 作成日時
        current_timestamp() AS created_datetime,
        -- 作成者
        NULL AS created_user,
        -- 更新日時
        current_timestamp() AS updated_datetime,
        -- 更新者
        NULL AS updated_user

        
    from
    (
        select
            
            'TOP60' as top_no,
            country as country,
            min(po_category) as po_category,
            sum(sales_qty_3w) as sales_qty_3w,
            sum(sales_qty_2w) as sales_qty_2w,
            sum(sales_qty_1w) as sales_qty_1w,
            sum(qty) as qty,
            sum(amount) as amount,
            sum(qty_last_year) as qty_last_year,
            sum(shop_inventory_qty) as shop_inventory_qty,
            sum(warehouse_inventory_qty) as warehouse_inventory_qty,
            sum(total_inventory_qty) as total_inventory_qty,
            sum(in_progress_stock) as in_progress_stock,
            sum(pre_month_order) as pre_month_order,
            sum(current_month_order) as current_month_order,
            sum(next_month_order) as next_month_order,
            sum(sf_newcomer_n_quantity) as newcomer_quantity,
            sum(sf_newcomer_quantity) as quantity,
            sum(woman_cnt) as customer_woman_cnt,
            sum(lens_uv_cnt) as uv_lens_cnt,
            sum(nonprescription_quantity) as nonprescription_quantity
        from
            top
        where
            rank <= 60
        group by
            'TOP60',
            country
    )

    union all
    select
        '04' as sort_number,
        top_no,
        country as country,
        '{week}' as weeks,
        '{report_from}' as report_from,
        '{report_to}' as report_to,
        sales_qty_3w,
        sales_qty_2w,
        sales_qty_1w,
        qty as qty,
        (qty/sales_qty_1w)*100 as qty_wow,
        qty - sales_qty_1w as qty_wow_diff,
        CASE
            when po_category = 'Proper Only' then qty/'{proper_all}'
            when po_category = 'All Store' then qty/'{both_all}' 
            when po_category = 'Outlet Only' then qty/'{outlet_all}' 
            else  0
        end as per_shop_qty,
        amount as amount,
        amount /'{total_amount}'*100 as revenue_ratio,
        '-' as cumulative_ratio,
        qty_last_year,
        (qty/qty_last_year)*100 as gross_yoy,
        CASE
                when po_category = 'Proper Only' then qty_last_year/'{proper_all_last_year}'
                when po_category = 'All Store' then qty_last_year/'{both_all_last_year}' 
                when po_category = 'Outlet Only' then qty_last_year/'{outlet_all_last_year}' 
                else  0
            end as per_shop_qty_last_year,
        CASE
                when po_category = 'Proper Only' then (qty/'{proper_all}')/(qty_last_year/'{proper_all_last_year}')*100
                when po_category = 'All Store' then (qty/'{both_all}')/(qty_last_year/'{both_all_last_year}')*100 
                when po_category = 'Outlet Only' then (qty/'{outlet_all}')/(qty_last_year/'{outlet_all_last_year}')*100 
                else  0
            end as per_shop_qty_yoy,
        shop_inventory_qty as shop_inventory_qty,
        shop_inventory_qty/qty as shop_inventory_qty_week,
        warehouse_inventory_qty as warehouse_inventory_qty,
        warehouse_inventory_qty/qty as warehouse_inventory_qty_week,
        total_inventory_qty as total_inventory_qty,
        total_inventory_qty/qty as total_inventory_qty_week,
        in_progress_stock as in_progress_stock,
        in_progress_stock/qty as in_progress_week,
        pre_month_order as pre_month_order,
        current_month_order as current_month_order,
        next_month_order as next_month_order,
        newcomer_quantity/quantity*100 as sf_newcomer_ratio,
        customer_woman_cnt/qty*100 as customer_woman_ratio,
        uv_lens_cnt/qty*100 as uv_lens_ratio,
        nonprescription_quantity/qty*100 as nonprescription_ratio,
        -- 作成日時
        current_timestamp() AS created_datetime,
        -- 作成者
        NULL AS created_user,
        -- 更新日時
        current_timestamp() AS updated_datetime,
        -- 更新者
        NULL AS updated_user
        
    from
    (
        select
            
            'TOP100' as top_no,
            country as country,
            min(po_category) as po_category,
            sum(sales_qty_3w) as sales_qty_3w,
            sum(sales_qty_2w) as sales_qty_2w,
            sum(sales_qty_1w) as sales_qty_1w,
            sum(qty) as qty,
            sum(amount) as amount,
            sum(qty_last_year) as qty_last_year,
            sum(shop_inventory_qty) as shop_inventory_qty,
            sum(warehouse_inventory_qty) as warehouse_inventory_qty,
            sum(total_inventory_qty) as total_inventory_qty,
            sum(in_progress_stock) as in_progress_stock,
            sum(pre_month_order) as pre_month_order,
            sum(current_month_order) as current_month_order,
            sum(next_month_order) as next_month_order,
            sum(sf_newcomer_n_quantity) as newcomer_quantity,
            sum(sf_newcomer_quantity) as quantity,
            sum(woman_cnt) as customer_woman_cnt,
            sum(lens_uv_cnt) as uv_lens_cnt,
            sum(nonprescription_quantity) as nonprescription_quantity
        from
            top
        where
            rank <= 100
        group by
            'TOP100',
            country
    )

    union all
    select
        '05' as sort_number,
        top_no,
        country as country,
        '{week}' as weeks,
        '{report_from}' as report_from,
        '{report_to}' as report_to,
        sales_qty_3w,
        sales_qty_2w,
        sales_qty_1w,
        qty,
        (qty/sales_qty_1w)*100 as qty_wow,
        qty - sales_qty_1w as qty_wow_diff,
        CASE
            when po_category = 'Proper Only' then qty/'{proper_all}'
            when po_category = 'All Store' then qty/'{both_all}' 
            when po_category = 'Outlet Only' then qty/'{outlet_all}' 
            else  0
        end as per_shop_qty,
        amount as amount,
        amount /'{total_amount}'*100 as revenue_ratio,
        '-' as cumulative_ratio,
        qty_last_year,
        (qty/qty_last_year)*100 as gross_yoy,
        CASE
                when po_category = 'Proper Only' then qty_last_year/'{proper_all_last_year}'
                when po_category = 'All Store' then qty_last_year/'{both_all_last_year}' 
                when po_category = 'Outlet Only' then qty_last_year/'{outlet_all_last_year}' 
                else  0
            end as per_shop_qty_last_year,
        CASE
                when po_category = 'Proper Only' then (qty/'{proper_all}')/(qty_last_year/'{proper_all_last_year}')*100
                when po_category = 'All Store' then (qty/'{both_all}')/(qty_last_year/'{both_all_last_year}')*100 
                when po_category = 'Outlet Only' then (qty/'{outlet_all}')/(qty_last_year/'{outlet_all_last_year}')*100 
                else  0
            end as per_shop_qty_yoy,
        shop_inventory_qty as shop_inventory_qty,
        shop_inventory_qty/qty as shop_inventory_qty_week,
        warehouse_inventory_qty as warehouse_inventory_qty,
        warehouse_inventory_qty/qty as warehouse_inventory_qty_week,
        total_inventory_qty as total_inventory_qty,
        total_inventory_qty/qty as total_inventory_qty_week,
        in_progress_stock as in_progress_stock,
        in_progress_stock/qty as in_progress_week,
        pre_month_order as pre_month_order,
        current_month_order as current_month_order,
        next_month_order as next_month_order,
        newcomer_quantity/quantity*100 as sf_newcomer_ratio,
        customer_woman_cnt/qty*100 as customer_woman_ratio,
        uv_lens_cnt/qty*100 as uv_lens_ratio,
        nonprescription_quantity/qty*100 as nonprescription_ratio,
        -- 作成日時
        current_timestamp() AS created_datetime,
        -- 作成者
        NULL AS created_user,
        -- 更新日時
        current_timestamp() AS updated_datetime,
        -- 更新者
        NULL AS updated_user

        
    from
    (
        select
            
            '総計' as top_no,
            country as country,
            min(po_category) as po_category,
            sum(sales_qty_3w) as sales_qty_3w,
            sum(sales_qty_2w) as sales_qty_2w,
            sum(sales_qty_1w) as sales_qty_1w,
            sum(qty) as qty,
            sum(amount) as amount,
            sum(qty_last_year) as qty_last_year,
            sum(shop_inventory_qty) as shop_inventory_qty,
            sum(warehouse_inventory_qty) as warehouse_inventory_qty,
            sum(total_inventory_qty) as total_inventory_qty,
            sum(in_progress_stock) as in_progress_stock,
            sum(pre_month_order) as pre_month_order,
            sum(current_month_order) as current_month_order,
            sum(next_month_order) as next_month_order,
            sum(sf_newcomer_n_quantity) as newcomer_quantity,
            sum(sf_newcomer_quantity) as quantity,
            sum(woman_cnt) as customer_woman_cnt,
            sum(lens_uv_cnt) as uv_lens_cnt,
            sum(nonprescription_quantity) as nonprescription_quantity
        from
            top
        group by
            '総計',
            country
    )
    order by
        sort_number
            
    """
    )

    gold_table = 'prd_im_dlh.gold.weekly_sales_ranking_report1_header_all_2'

    # ゴールドテーブルにデータを挿入
    header_df.write.insertInto(gold_table, overwrite=False)

    display(header_df)

# エラー処理
except Exception as e:
    # ログを出力する
    logger = setup_logging(logtype="error")
    logger.error(f"Spark session 処理中にエラーが発生しました: {e}")